In [86]:
import pandas as pd
import numpy as np  
import openpyxl 
import re

In [87]:
noticias = pd.read_csv("C:\\Users\\juans\\Desktop\\Analisis de datos\\Proyecto Fake News\\FakeNews\\Web_Scrapping\\noticias_consolidadas.csv")
noticias.head()

,url,title,subtitle,body,date_published,author,image,section,tags,medio,espectro_politico
0,https://www.elnuevosiglo.com.co/politica/sigue...,Siguen movidas alrededor de procesos contra Ni...,NaN,UNA SORPRESIVA solicitud hizo por medio de su ...,"Mar, 08/08/2023 - 21:20",admin,https://www.elnuevosiglo.com.co/sites/default/...,politica,NaN,ElNuevoSiglo,derecha
1,https://www.elnuevosiglo.com.co/politica/minis...,“Ministro que no ejecute tendrá menos presupue...,NaN,UNA DURO reparo hizo ayer el Congreso de la Re...,"Mar, 08/08/2023 - 20:15",admin,https://www.elnuevosiglo.com.co/sites/default/...,politica,NaN,ElNuevoSiglo,derecha
2,https://www.elnuevosiglo.com.co/columnistas/la...,Las nuevas tecnologías y la política,NaN,"Para que el Estado -en particular, si se procl...","Mar, 08/08/2023 - 19:26",admin,https://www.elnuevosiglo.com.co/sites/default/...,politica,NaN,ElNuevoSiglo,derecha
3,https://www.elnuevosiglo.com.co/columnistas/la...,Las nuevas tecnologías y la política,NaN,"Desborda la imaginación, el nexo e impacto en ...","Mar, 08/08/2023 - 19:24",admin,https://www.elnuevosiglo.com.co/sites/default/...,politica,NaN,ElNuevoSiglo,derecha
4,https://www.elnuevosiglo.com.co/politica/el-pr...,El problema es la fórmula de cálculo del combu...,NaN,ANTE el sucesivo incremento del valor de la ga...,"Mar, 08/08/2023 - 15:53",admin,https://www.elnuevosiglo.com.co/sites/default/...,politica,NaN,ElNuevoSiglo,derecha


In [88]:
datos_limpios = noticias.copy()
datos_limpios = datos_limpios.drop(columns=[ 'url', 'image', 'subtitle', 'date_published', 'author', 'tags'])

datos_limpios[datos_limpios['body'].isnull()]
datos_limpios = datos_limpios.dropna(subset=['body'])

""" reemplazar los caracteres especiales por vocales usando regexp y eliminar los espacios en blanco extras """
def limpiar_texto(texto):
    texto = re.sub(r'á', 'a', texto)
    texto = re.sub(r'é', 'e', texto)
    texto = re.sub(r'í', 'i', texto)
    texto = re.sub(r'ó', 'o', texto)
    texto = re.sub(r'ú', 'u', texto)
    texto = re.sub(r'Á', 'A', texto)
    texto = re.sub(r'É', 'E', texto)
    texto = re.sub(r'Í', 'I', texto)
    texto = re.sub(r'Ó', 'O', texto)
    texto = re.sub(r'Ú', 'U', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

datos_limpios['body'] = datos_limpios['body'].apply(limpiar_texto)
datos_limpios['title'] = datos_limpios['title'].apply(limpiar_texto)

""" retirar todas las palabras que contengan caracteres especiales"""

def eliminar_caracteres_especiales(texto):
    palabras = texto.split()
    palabras_limpias = [palabra for palabra in palabras if re.match(r'^[a-zA-Z0-9áéíóúÁÉÍÓÚ]+$', palabra)]
    return ' '.join(palabras_limpias)
datos_limpios['body'] = datos_limpios['body'].apply(eliminar_caracteres_especiales)
datos_limpios['title'] = datos_limpios['title'].apply(eliminar_caracteres_especiales)

datos_limpios['body'] = datos_limpios['body'].str.lower()
datos_limpios['title'] = datos_limpios['title'].str.lower()

datos_limpios = datos_limpios[(datos_limpios['body'].str.len() > 100) & (datos_limpios['title'].str.len() > 20)]

datos_limpios

,title,body,section,medio,espectro_politico
0,siguen movidas alrededor de procesos contra ni...,una sorpresiva solicitud hizo por medio de su ...,politica,ElNuevoSiglo,derecha
1,que no ejecute tendra menos congreso,una duro reparo hizo ayer el congreso de la re...,politica,ElNuevoSiglo,derecha
2,las nuevas tecnologias y la politica,para que el estado si se proclama tenga sentid...,politica,ElNuevoSiglo,derecha
3,las nuevas tecnologias y la politica,desborda la el nexo e impacto en ciertos nicho...,politica,ElNuevoSiglo,derecha
4,el problema es la formula de calculo del pedraza,ante el sucesivo incremento del valor de la ga...,politica,ElNuevoSiglo,derecha
...,...,...,...,...,...
1229,rayos x a las cuotas politicas en amazonas,un un representante a la camara y un politico ...,politica,LaSillaVacia,centro
1230,cambio radical propone hundir la reforma politica,el unico ponente de este partido dice que el p...,politica,LaSillaVacia,centro
1232,cada quien se lleva un pedazo de la reforma po...,el proyecto que arranca hoy su debate en el co...,politica,LaSillaVacia,centro
1233,los que se bajaron del bus de la reforma politica,diferentes personajes han ido mostrando su rec...,politica,LaSillaVacia,centro


In [89]:
datos_limpios.espectro_politico.value_counts()

espectro_politico
centro       525
derecha      317
izquierda    298
Name: count, dtype: int64

In [94]:
datos_limpios.to_csv("C:\\Users\\juans\\Desktop\\Analisis de datos\\Proyecto Fake News\\FakeNews\\Web_Scrapping\\noticias_limpias.csv", index=False)